In [ ]:
%%time
import json
import inspect
import tqdm
import contractions

currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
os.sys.path.insert(0,parentdir) 

import Artha
from Artha.analysis import *
from Artha.nlp_extraction import *


In [ ]:
%%time
user = "Nostranomist"
# user = "CryptoKaleo"
with open("../data/tweets/u"+user+"tweets.json", "r") as r:
    tweets = json.load(r)
tweet_text = []
for ind, tweet in enumerate(tweets):
    sent = tweet["full_text"]
    
    try: #remove tweets of only tagging someone
        while sent[0] == "@":
            sent = sent.split(" ", 1)[1]
        tweet_text.append(contractions.fix(sent.replace("&amp;", "and").replace("@", "")))
    except:
        tweets.pop(ind)

In [ ]:
%%time
# need to disable pipes
with open("senti.txt", "w+") as w:
    for ind, doc in enumerate(nlp.pipe(tweet_text, n_process=-1)):
        tick = doc._.tickers
                
        if tick:
            w.write("***"+str(ind)+" "+" ".join(tick)+"***"+str(doc._.polarity)+" "+str(doc._.subjectivity)+"\n")
            w.write(doc.text+"\n\n")

In [ ]:
doc = nlp(tweet_text[1883])
print(sorted(doc._.tickers))

for ent in doc.ents:
    print(ent, ent.label_)